# Извлечение признаков из текста

Этот блокнот разделён на две части:
* Для начала - выясним, что понадобится для построения инструментария NLP, который превратит набор текста в числовой массив из признаков. Для этого мы вручную вычислим, как часто встречаются те или иные слова, и построим метрику TF-IDF.
* Далее выполним эти шаги с помощью Scikit-Learn.

# Часть 1: основные принципы извлечения признаков из текста
В этой части мы с помощью базовых операций Python построим очень простую систему NLP. Возьмём набор документов - это будет два текстовых файла. Далее создадим словарь из всех слов обоих документов. И затем посмотрим на технику **Мешок слов (Bag of Words)** для извлечения признаков из каждого документа.

<div class="alert alert-info" style="margin: 20px">В этом разделе мы только приводим иллюстрации основных принципов!
<br>Не обращайте внимание на то, как пишется код в Python - позднее мы применим для этой задачи Scikit-Learn.</div>

In [75]:
import numpy as np
import pandas as pd

## Начнём с документов
Для простоты изложения в наших текстовых файлах One.txt и Two.txt не будет каких-либо знаков пунктуации. Для начала откроем эти файлы и прочитаем данные. **Важно:** если в будущем файлы будут большие, то их не следует выводить на экран полностью.

In [2]:
with open('One.txt') as mytext:
    print(mytext.read())

This is a story about dogs
our canine pets
Dogs are furry animals



In [4]:
with open('Two.txt') as mytext:
    print(mytext.read())

This story is about surfing
Catching waves is fun
Surfing is a popular water sport



### Читаем весь текст как строку string

In [6]:
with open('One.txt') as mytext:
    a = mytext.read()

In [8]:
a

'This is a story about dogs\nour canine pets\nDogs are furry animals\n'

In [10]:
print(a)

This is a story about dogs
our canine pets
Dogs are furry animals



### Читаем каждую строку файла отдельно и помещаем в список

In [12]:
with open('One.txt') as mytext:
    lines = mytext.readlines()

In [14]:
lines

['This is a story about dogs\n',
 'our canine pets\n',
 'Dogs are furry animals\n']

### Читаем отдельные слова

In [16]:
with open('One.txt') as f:
    words = f.read().lower().split()

In [18]:
words

['this',
 'is',
 'a',
 'story',
 'about',
 'dogs',
 'our',
 'canine',
 'pets',
 'dogs',
 'are',
 'furry',
 'animals']

## Создаём словарь vocabulary (Мешок слов - "Bag of Words")
Для этого мы возьмём все слова из обоих документов, найдём уникальные слова, и пронумеруем их.

In [22]:
with open('One.txt') as mytext:
    words_one = mytext.read().lower().split()

In [24]:
words_one

['this',
 'is',
 'a',
 'story',
 'about',
 'dogs',
 'our',
 'canine',
 'pets',
 'dogs',
 'are',
 'furry',
 'animals']

In [26]:
len(words_one)

13

**Получим набор уникальных значений**

In [28]:
uni_words_one = set(words_one)

In [30]:
uni_words_one

{'a',
 'about',
 'animals',
 'are',
 'canine',
 'dogs',
 'furry',
 'is',
 'our',
 'pets',
 'story',
 'this'}

In [32]:
with open('Two.txt') as mytext:
    words_two = mytext.read().lower().split()
    uni_words_two = set(words_two)

In [34]:
uni_words_two

{'a',
 'about',
 'catching',
 'fun',
 'is',
 'popular',
 'sport',
 'story',
 'surfing',
 'this',
 'water',
 'waves'}

**Получаем все уникальные слова из всех документов**

In [37]:
all_uni_words = set()
all_uni_words.update(uni_words_one)
all_uni_words.update(uni_words_two)

In [39]:
all_uni_words

{'a',
 'about',
 'animals',
 'are',
 'canine',
 'catching',
 'dogs',
 'fun',
 'furry',
 'is',
 'our',
 'pets',
 'popular',
 'sport',
 'story',
 'surfing',
 'this',
 'water',
 'waves'}

**Пронумеруем все слова.**

В цикле пройдём по всем словам из `all_uni_words`. Для каждого слова добавим элемент в словарь `full_vocab`, где ключом будет само слово, а значением - очередное число.

In [43]:
full_vocab = {}
i=0

for word in all_uni_words:
    full_vocab[word] = i
    i = i+1

In [45]:
full_vocab

{'story': 0,
 'pets': 1,
 'furry': 2,
 'about': 3,
 'animals': 4,
 'our': 5,
 'water': 6,
 'waves': 7,
 'dogs': 8,
 'is': 9,
 'are': 10,
 'popular': 11,
 'catching': 12,
 'sport': 13,
 'fun': 14,
 'canine': 15,
 'surfing': 16,
 'a': 17,
 'this': 18}

**Множества set() - не являются отсортированными! Поэтому порядок элементов разный.**

## Мешок слов - считаем, как часто встречаются отдельные слова
После того, как словарь из слов создан, выполним извлечение признаков для каждого из двух исходных документов.

**Создаём пустые счётчики для каждого документа**

In [49]:
# Список будет содержать столько нулей,
# сколько элементов содержится в словаре full_vocab
one_freq = [0] * len(full_vocab)
two_freq = [0] * len(full_vocab)

# В этом списке вместо 0 будут пустые строковые значения
all_words = [''] * len(full_vocab)

In [51]:
one_freq

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [53]:
two_freq

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [55]:
all_words

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

**Добавляем счётчики слов для каждого документа**

Теперь пройдём по файлу One.txt и для каждого слова будем находить соответствующий этому слову номер в словаре `full_vocab`, а далее, будем увеличивать на единицу счётчик в списке `one_freq`. Таким образом будем подсчитывать сколько раз то или иное слово встречается в файле One.txt.

Затем выполним всё то же самое для файла Two.txt, после чего объединим все слова из обоих файлов.

In [60]:
with open('One.txt') as f:
    one_text = f.read().lower().split()

for word in one_text:
    # Для каждого слова найдём его индекс в словаре
    word_ind = full_vocab[word]
    one_freq[word_ind] += 1

В переменной `one_text` содержится набор слов из файла One.txt. В цикле, мы берём каждое из этих слов и находим это слово в словаре `full_vocab`. Далее в списке со счётчиком увеличивается значение счётчика, соответствующего номеру текущего слова в словаре.

In [63]:
one_freq

[1, 1, 1, 1, 1, 1, 0, 0, 2, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1]

In [65]:
with open('Two.txt') as f:
    two_text = f.read().lower().split()

for word in two_text:
    # Для каждого слова найдём его индекс в словаре
    word_ind = full_vocab[word]
    two_freq[word_ind] += 1

In [67]:
two_freq

[1, 0, 0, 1, 0, 0, 1, 1, 0, 3, 0, 1, 1, 1, 1, 0, 2, 1, 1]

**Мы получили счётчики слов для двух файлов. Теперь можно создать список со всеми словами.**

In [71]:
for word in full_vocab:
     word_ind = full_vocab[word]
     # Здесь уже не увеличение счётчика, а просто записываем слово
     all_words[word_ind] = word

In [73]:
all_words

['story',
 'pets',
 'furry',
 'about',
 'animals',
 'our',
 'water',
 'waves',
 'dogs',
 'is',
 'are',
 'popular',
 'catching',
 'sport',
 'fun',
 'canine',
 'surfing',
 'a',
 'this']

In [79]:
bow = pd.DataFrame(data=[one_freq, two_freq], columns=all_words)

In [81]:
bow

,story,pets,furry,about,animals,our,water,waves,dogs,is,are,popular,catching,sport,fun,canine,surfing,a,this
0,1,1,1,1,1,1,0,0,2,1,1,0,0,0,0,1,0,1,1
1,1,0,0,1,0,0,1,1,0,3,0,1,1,1,1,0,2,1,1


Некоторые слова есть в обоих документах, но также есть слова которые находятся только в One.txt, а другие только в Two.txt. Если применить эту логику для десятков тысяч документов, то наш словарь вполне сможет вырасти до десятков тысяч слов. При этом матрицы будут содержать очень много нулей - это будут разреженные метрицы (sparse matrices).

# Продолжение:

## Мешок слов и Tf-idf
В приведённом выше примере, каждый вектор можно рассматривать как мешок слов (*bag of words*). Сами по себе эти вектора не очень полезны, но мы также можем добавить к ним частоту слов (*term frequencies - TF*) - как часто то или иное слово встречается в документе. Простой способ сделать это - это посчитать, сколько раз встречается слово в документе, и разделить на общее количество слов в документе. Тогда можно сравнивать между большими и маленькими документами, сколько раз то или иное слово встречается в документе.

Однако, если какое-то слово встречается во многих документах, то такое слово не будет являться хорошим признаком для отделения документов друг от друга. Чтобы работать с такими словами, можно добавить метрику *inverse document frequency (IDF)*, которая вычисляется как общее количество документов, разделить на количество документов, в которых содержится рассматриваемое нами слово (слово, для которого вычисляется IDF). В практических задачах это значение приводится к логарифмической шкале, подробнее см. [здесь](https://ru.wikipedia.org/wiki/TF-IDF).

Соединяя метрики TF (Term Frequency) и IDF (Inverse Document Frequency), мы получаем метрику [**tf-idf**](https://ru.wikipedia.org/wiki/TF-IDF).

## Стоп-слова и морфология слов
Некоторые слова встречаются слишком часто, например слова "the" и "and" в английском языке. Эти слова можно просто исключить из рассмотрения. 

Кроме этого, одно и то же слово может встречаться в единственном и множественном числе, а также в различных падежах. Нам бы хотелось записать это слово один раз, а не записывать различные его вариации (например, записать только `cat` для обоих слов `cat` и `cats`). Это позволит нам уменьшить размер словаря и увеличить скорость работы модели.

С другой стороны, различные падежи слов - особенно в русском языке - несут дополнительную смысловую информацию. Эту информацию, по-хорошему, можно сохранять, то есть ссылаться на базовое слово в словаре, плюс сохранять дополнительные атрибуты этого слова в конкретном месте текста. В данном блокноте мы ограничимся лишь базовым подсчётом частоты слов - как в отдельном документе, так и в целом наборе документов.

# Часть 2: извлечение признаков и текста в Scikit-Learn

# Варианты извлечения признаков в Scikit-Learn

In [90]:
text = ["This is a line",
        "This is another line",
        "Completely different line"]

## CountVectorizer

In [93]:
from sklearn.feature_extraction.text import CountVectorizer

In [95]:
help(CountVectorizer)

Help on class CountVectorizer in module sklearn.feature_extraction.text:

class CountVectorizer(_VectorizerMixin, sklearn.base.BaseEstimator)
 |  CountVectorizer(*, input='content', encoding='utf-8', decode_error='strict', strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), analyzer='word', max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class 'numpy.int64'>)
 |
 |  Convert a collection of text documents to a matrix of token counts.
 |
 |  This implementation produces a sparse representation of the counts using
 |  scipy.sparse.csr_matrix.
 |
 |  If you do not provide an a-priori dictionary and you do not use an analyzer
 |  that does some kind of feature selection then the number of features will
 |  be equal to the vocabulary size found by analyzing the data.
 |
 |  For an efficiency comparison of the different feature extractors, see
 |  :ref:`sphx_glr_auto_examp

`CountVectorizer` - подсчитывает, сколько раз то или иное слово встречается в документе. 

Параметр `stop_words` позволяет удалить из рассмотрения слишком часто встречающиеся слова. По умолчанию установлено значение None, можно передать как например *english*, так и список собственных слов.

In [98]:
cv = CountVectorizer()

In [101]:
cv.fit_transform(text)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 10 stored elements and shape (3, 6)>

В данном случае, каждая из трёх строк будет обрабатываться как отдельный документ. На выходе мы получаем разреженную матрицу размером 3 на 6 (18 элементов), в которой находится 10 **ненулевых** чисел в сжатом формате. Здесь 3 - это количество документов, в нашем случае - это три строки, а 6 - это количество различных слов во всех этих документах.

In [105]:
sparse_matrix = cv.fit_transform(text)

**Далее эту матрицу можно преобразовать из разреженной в обычную с помощью метода `todense()`.**

**Важно: при работе с большими разреженными матрицами - лучше не запускать метод `todense()`.**

In [108]:
sparse_matrix.todense()

matrix([[0, 0, 0, 1, 1, 1],
        [1, 0, 0, 1, 1, 1],
        [0, 1, 1, 0, 1, 0]])

**Посмотрим словарь слов**

In [112]:
cv.vocabulary_

{'this': 5, 'is': 3, 'line': 4, 'another': 0, 'completely': 1, 'different': 2}

Это такой же словарь, как и тот, что был создан нами вручную ранее. Например, для слова *another* - индекс 0, встречается оно только во втором документе - единица во втором списке на первой(нулевой) позиции, а в первой и третьей строке для этого слова указаны 0.

## Стоп-слова

In [117]:
cv = CountVectorizer(stop_words='english')

In [119]:
sparse_matrix = cv.fit_transform(text)

In [121]:
sparse_matrix.todense()

matrix([[0, 0, 1],
        [0, 0, 1],
        [1, 1, 1]])

In [123]:
cv.vocabulary_

{'line': 2, 'completely': 0, 'different': 1}

Мы получили матрицу размером 3 на 3. В словаре также осталось только 3 слова. Из рассмотрения были исключены такие слова как: this, is, another.

## TfidfTransformer

Посмотрим как можно преобразовать данные с помощью TF-IDF, т.е. не только подсчитать количество слов с помощью TF, но и с помощью IDF учесть то, насколько часто слова встречаются во всём наборе документов.

`TfidfVectorizer` применяется **для текстовых документов**, а `TfidfTransformer` применяется **к матрице со счётчиками, которую возвращает CountVectorizer**

In [127]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

In [129]:
tfidf = TfidfTransformer()

**Рассмотрим полный словарь из 6 слов, чтобы было нагляднее**

In [132]:
cv = CountVectorizer()

In [134]:
counts = cv.fit_transform(text)

In [136]:
counts

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 10 stored elements and shape (3, 6)>

In [139]:
results = tfidf.fit_transform(counts)

Здесь мы на вход подаём мешок слов, а на выходе получаем значение TF-IDF.

In [141]:
results

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10 stored elements and shape (3, 6)>

In [143]:
results.todense()

matrix([[0.        , 0.        , 0.        , 0.61980538, 0.48133417,
         0.61980538],
        [0.63174505, 0.        , 0.        , 0.4804584 , 0.37311881,
         0.4804584 ],
        [0.        , 0.65249088, 0.65249088, 0.        , 0.38537163,
         0.        ]])

Мы получили счётчики TF-IDF, но уже в два шага. Сначала был запущен `CountVectorizer`, а затем `TfidfTransformer`. Но эти два шага можно объединить в один, с помощью `TfidfVectorizer`.

## TfIdfVectorizer

In [148]:
tv = TfidfVectorizer()

In [150]:
tv.fit_transform(text)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10 stored elements and shape (3, 6)>

In [152]:
tv_results = tv.fit_transform(text)

In [154]:
tv_results.todense()

matrix([[0.        , 0.        , 0.        , 0.61980538, 0.48133417,
         0.61980538],
        [0.63174505, 0.        , 0.        , 0.4804584 , 0.37311881,
         0.4804584 ],
        [0.        , 0.65249088, 0.65249088, 0.        , 0.38537163,
         0.        ]])

`TfidfVectorizer` возвращает уже готовые результаты. Однако иногда может возникнуть вопрос по отдельным элементам, почему результаты получились именно такими, какими получились - например, подозрительно большими или подозрительно маленькими. Тогда можно использовать промежуточные методы - прежде всего `CountVectorizer` - чтобы посмотреть промежуточные результаты и понять, как именно получились результаты для `TfidfVectorizer`.

In [157]:
words_dict = {k: v for k, v in sorted(tv.vocabulary_.items(), key=lambda item: item[1])}

In [159]:
pd.DataFrame(tv_results.todense(), columns=words_dict)

,another,completely,different,is,line,this
0,0.000000,0.000000,0.000000,0.619805,0.481334,0.619805
1,0.631745,0.000000,0.000000,0.480458,0.373119,0.480458
2,0.000000,0.652491,0.652491,0.000000,0.385372,0.000000
